In [2]:
import pandas as pd
import numpy as nb

In [3]:
#load csv
heroes = "purchase_data_3.csv"

In [4]:
#read with pandas
heroes_pd = pd.read_csv(heroes, encoding="iso-8859-1", low_memory=False)
heroes_pd.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lirtjaskan85,24,Female,69,"Frenzy, Defender of the Harvest",4.82
1,1,Chanjask65,12,Female,75,Brutality Ivory Warmace,4.12
2,2,Aerithllora36,21,Female,114,Yearning Mageblade,2.67
3,3,Aeralria27,24,Male,130,Alpha,4.53
4,4,Haisrisuir60,28,Male,9,"Thorn, Conqueror of the Corrupted",4.60


In [5]:
# Calculate total number of players
players = heroes_pd['SN'].nunique()
players

581

In [6]:
#Purchasing Analysis
#find the variables first and then put in summary table
average_price = heroes_pd["Price"].mean()
items = heroes_pd['Item Name'].nunique()
purchases = len(heroes_pd)
revenue = heroes_pd['Price'].sum()
#create summary table
purchasing_summary = pd.DataFrame({"Number of Unique Items": [items],
                                   "Average Price": [average_price],
                                   "Number of Purchases": [purchases],
                                   "Total Revenue": [revenue]
})
purchasing_summary = purchasing_summary.round(2)
purchasing_summary

,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
0,3.03,780,178,2365.17


In [7]:
#Gender Demographics
#Find amount of men and women players and other / this works
grouped_players= heroes_pd.groupby(['SN'], as_index=False).count()

#find men, and women, they are 1 and 2 in grouped players...minus from players to find others or null values
men = grouped_players["Gender"].value_counts()[1]
women = grouped_players["Gender"].value_counts()[2]
other = players - men - women

#find percents
Men_Percent = men/players * 100
Women_Percent = women /players * 100
Other_Percent = other /players * 100

gender_summary = pd.DataFrame({"Gender": ["Males", "Females", "Other"],                    
                               "Percentage of Players": [Men_Percent, Women_Percent, Other_Percent],
                               "Total Count": [men, women, other]
})
gender_summary = gender_summary.round(2)
gender_summary.head()

,Gender,Percentage of Players,Total Count
0,Males,70.91,412
1,Females,24.61,143
2,Other,4.48,26


In [8]:
#Purchasing Analysis (Gender)
male_group = heroes_pd[heroes_pd["Gender"] == "Male"]

men_items = len(male_group)
men_average = male_group["Price"].mean()
mentotal =  male_group["Price"].sum()
mennormal = men_average - average_price

#first filter out for females and then 
female_group = heroes_pd[heroes_pd["Gender"] == "Female"]

fem_items = len(female_group)
fem_average = female_group["Price"].mean()
femtotal =  female_group["Price"].sum()
femnormal = men_average - average_price

other_group = heroes_pd[heroes_pd["Gender"]=="Other / Non-Disclosed"]
other_items = len(other_group)
other_average = other_group["Price"].mean()
othertotal =  other_group["Price"].sum()
othernormal = other_average - average_price

gender_purchasing = pd.DataFrame({"Gender": ["Males", "Females", "Other"],
                                   "Purchase Count": [men_items, fem_items, other_items],
                                   "Average Purchase Price": [men_average, fem_average, other_average],
                                   "Total Purchase Value": [mentotal, femtotal, othertotal],
                                   "Normalized Average": [mennormal, femnormal, othernormal]
})

gender_purchasing = gender_purchasing.round(2)
gender_purchasing

,Average Purchase Price,Gender,Normalized Average,Purchase Count,Total Purchase Value
0,3.03,Males,-0.00,642,1945.51
1,3.04,Females,-0.00,130,395.80
2,2.98,Other,-0.05,8,23.86


In [9]:
#Age Demographics
#create bins
bins = [0, 9, 14, 19, 24, 29, 34, 39,150]

# Create the names for bins
group_labels = ['<10','10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

AgeGroup = pd.cut(heroes_pd["Age"],bins,labels=group_labels)
heroes_pd["Age Group"] = pd.cut(heroes_pd["Age"],bins,labels=group_labels)

age_count = pd.DataFrame(pd.value_counts(heroes_pd['Age Group']))
age_count

,Age Group
20-24,316
25-29,137
15-19,132
30-34,62
35-39,54
<10,37
10-14,31
40+,11


In [10]:
#Age Analysis
groupby_age = heroes_pd['Price'].groupby(heroes_pd['Age Group'])
groupby_analysis = pd.DataFrame(heroes_pd['Price'].groupby(heroes_pd['Age Group']).describe())
groupby_analysis 
groupby_analysis = groupby_analysis.drop('std', 1)
groupby_analysis = groupby_analysis.drop('min', 1)
groupby_analysis = groupby_analysis.drop('25%', 1)
groupby_analysis = groupby_analysis.drop('50%', 1)
groupby_analysis = groupby_analysis.drop('75%', 1)
groupby_analysis = groupby_analysis.drop('max', 1)
groupby_analysis
groupby_analysis["Total Purchase Value"] = groupby_analysis["count"] * groupby_analysis["mean"]
groupby_analysis.round(2)
groupby_analysis.rename(index=str, columns={"count": "Purchase Count", "mean": "Average Purchase Value"})


,Purchase Count,Average Purchase Value,Total Purchase Value
Age Group,,,
<10,37.0,2.917838,107.96
10-14,31.0,2.981290,92.42
15-19,132.0,3.080455,406.62
20-24,316.0,3.008196,950.59
25-29,137.0,3.045328,417.21
30-34,62.0,2.925806,181.40
35-39,54.0,3.175926,171.50
40+,11.0,3.406364,37.47


In [11]:
#Top 5 Spenders

sn_price = heroes_pd.groupby(["SN"])['Price'].sum()
sn_pur = heroes_pd.groupby(["SN"])['Price'].count()
sn_users = heroes_pd.groupby(["SN"])
avg_sn = sn_price / sn_pur

top_sn = pd.DataFrame({"Purchase Count": sn_pur,"Average Purchase Price":avg_sn, "Total Purchase Value": sn_price})

top_sn = top_sn.sort_values("Total Purchase Value", ascending=False)
top_sn = top_sn[["Purchase Count","Average Purchase Price", "Total Purchase Value"]]

top_sn.reset_index(inplace=True)
top_sn.round(2)
top_sn.head()

,SN,Purchase Count,Average Purchase Price,Total Purchase Value
0,Chamalo71,4,3.362500,13.45
1,Strithenu87,4,3.207500,12.83
2,Mindosia50,3,4.003333,12.01
3,Aeralria27,3,3.793333,11.38
4,Eudai71,3,3.790000,11.37


In [12]:
#Items by Total Price
item_price = heroes_pd.groupby(["Item Name"])['Price'].sum()
item_num = heroes_pd.groupby(["Item Name"])['Price'].count()
item_name = heroes_pd.groupby(["Item Name"])
avg_item = item_price / item_num

best_item = pd.DataFrame({"Purchase Count": item_num,"Average Purchase Price":avg_item, "Total Purchase Value": item_price})

best_item = best_item.sort_values("Total Purchase Value", ascending=False)
best_item = best_item[["Purchase Count","Average Purchase Price", "Total Purchase Value"]]

best_item.reset_index(inplace=True)
best_item.round(2)
best_item.head()

,Item Name,Purchase Count,Average Purchase Price,Total Purchase Value
0,Hatred,11,4.59,50.49
1,Misery's End,9,4.90,44.10
2,Agatha,8,4.93,39.44
3,Apocalyptic Battlescythe,8,4.85,38.80
4,"The Oculus, Token of Lost Worlds",8,4.61,36.88


In [13]:
#Items by Number of Purchases
item_price = heroes_pd.groupby(["Item Name"])['Price'].sum()
item_num = heroes_pd.groupby(["Item Name"])['Price'].count()
item_name = heroes_pd.groupby(["Item Name"])
avg_item = item_price / item_num

best_item = pd.DataFrame({"Purchase Count": item_num,"Average Purchase Price":avg_item, "Total Purchase Value": item_price})

best_item = best_item.sort_values("Purchase Count", ascending=False)
best_item = best_item[["Purchase Count","Average Purchase Price", "Total Purchase Value"]]

best_item.reset_index(inplace=True)
best_item.round(2)
best_item.head()

,Item Name,Purchase Count,Average Purchase Price,Total Purchase Value
0,Hatred,11,4.59,50.49
1,Final Critic,11,2.70,29.70
2,Primitive Blade,9,1.39,12.51
3,Misery's End,9,4.90,44.10
4,"The Oculus, Token of Lost Worlds",8,4.61,36.88
